# 📘 Data Merging & Cleaning – Yousra Descriptive Stats Notebook

This notebook loads the original project datasets (sales, weather, kiwo event),  
cleans and standardizes the date format, merges them using a full outer join,  
and performs descriptive statistics, missing value inspection, and prepares the  
data for further feature engineering and modeling.

The goal is to:
1. Combine **all available data** on matching dates  
2. Perform a clear descriptive exploration  
3. Detect and handle missing values  
4. Produce a clean dataset ready for analysis  


In [ ]:
import pandas as pd
import numpy as np

# Load data from your project folder
df_kiwo = pd.read_csv("data/kiwo.csv")
df_weather = pd.read_csv("data/wetter.csv")
df_umsatz = pd.read_csv("data/umsatzdaten_gekuerzt.csv")

df_kiwo.head(), df_weather.head(), df_umsatz.head()
#print number of rows for each dataframe
print(f"Kiwo Data Rows: {len(df_kiwo)}")
print(f"Weather Data Rows: {len(df_weather)}")
print(f"Umsatz Data Rows: {len(df_umsatz)}")

In [ ]:
# Convert 'Datum' to proper datetime format and drop invalid rows
for df in (df_kiwo, df_weather, df_umsatz):
    df["Datum"] = pd.to_datetime(df["Datum"], errors="coerce").dt.normalize()
    df.dropna(subset=["Datum"], inplace=True)

    # Drop duplicate date columns if exist
    if "date" in df.columns:
        df.drop(columns=["date"], inplace=True)

df_kiwo.info(), df_weather.info(), df_umsatz.info()


In [ ]:
# Adding suffixes helps avoid confusion during merging

df_kiwo = df_kiwo.add_suffix("_kiwo")
df_kiwo.rename(columns={"Datum_kiwo": "Datum"}, inplace=True)

df_weather = df_weather.add_suffix("_weather")
df_weather.rename(columns={"Datum_weather": "Datum"}, inplace=True)


df_umsatz = df_umsatz.add_suffix("_umsatz")
df_umsatz.rename(columns={"Datum_umsatz": "Datum"}, inplace=True)

df_umsatz["umsatz_rolling7"] = (
    df_umsatz["Umsatz_umsatz"].rolling(window=7, min_periods=1).mean()
)


In [ ]:
merged_df = (
    df_kiwo
    .merge(df_weather, on="Datum", how="outer")
    .merge(df_umsatz, on="Datum", how="outer")
)

merged_df.shape
merged_df.head()


In [ ]:
# Numeric summary
merged_df.describe()

# For all columns (including categorical)
merged_df.describe(include="all")


In [ ]:
missing_count = merged_df.isna().sum()
missing_percent = (merged_df.isna().sum() / len(merged_df)) * 100

missing_df = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing %": missing_percent
})

missing_df


### ✔️ Missing Value Strategy

**Umsatz (Sales):**
- Missing values mean the day has **no sales** or is **outside the bakery event period**.
- These rows should be **removed**, not imputed.

**Weather Variables:**
- Weather data may be missing because not all dates have weather records.
- Fill missing values using interpolation (numerical)  
  and assign -1 for Wettercode to indicate “unknown weather”.

**Kiwo Event Flag:**
- If missing → fill with 0 (day outside event).


In [ ]:
clean_df = merged_df.copy()

# Remove rows where sales are missing
clean_df = clean_df.dropna(subset=["Umsatz_umsatz"])

# Fill missing Wettercode with category -1
if "Wettercode_weather" in clean_df.columns:
    clean_df["Wettercode_weather"] = clean_df["Wettercode_weather"].fillna(-1)

# Interpolate numerical weather values
for col in clean_df.columns:
    if ("_weather" in col) and (clean_df[col].dtype in ["float64", "int64"]):
        clean_df[col] = clean_df[col].interpolate()

clean_df.isna().sum()


In [ ]:
clean_df["KielerWoche_kiwo"] = clean_df["KielerWoche_kiwo"].fillna(0)


In [ ]:
clean_df.isna().sum()


In [ ]:
#drop wettercode_weather column
df_no_weather_code = clean_df.drop(columns=["Wettercode_weather"])
df_no_weather_code.sample(5)


## Adding extra columns

In [ ]:
df_no_weather_code['Datum'] = pd.to_datetime(df_no_weather_code['Datum'], errors='coerce').dt.normalize()
n_invalid = df_no_weather_code['Datum'].isna().sum()
if n_invalid > 0:
    print(f"Warning: {n_invalid} rows have invalid 'Datum' and will have NaT in day columns. Sample:")
    display(df_no_weather_code[df_no_weather_code['Datum'].isna()].head())

# Create integer and name columns for day of week
df_no_weather_code['day_of_week'] = df_no_weather_code['Datum'].dt.weekday  # Monday=0 .. Sunday=6
df_no_weather_code['day'] = df_no_weather_code['Datum'].dt.day_name()

# Optional: make 'day' categorical ordered Monday..Sunday
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_no_weather_code['day'] = pd.Categorical(df_no_weather_code['day'], categories=ordered_days, ordered=True)

print("\nCounts per weekday:")
print(df_no_weather_code['day'].value_counts().sort_index())
df_no_weather_code.sample(5)

### Adding more weather data from meteo archive api

In [ ]:
import requests
from datetime import datetime, date, timedelta
import time
from typing import Union, Iterable, Dict, List
import pandas as pd
import numpy as np

# Simple in-memory cache for API responses
try:
    _OPEN_METEO_CACHE
except NameError:
    _OPEN_METEO_CACHE: Dict[str, Dict[str, float]] = {}

def _iso_date(d: Union[str, date, datetime]) -> str:
    if isinstance(d, str):
        return datetime.fromisoformat(d).date().isoformat()
    if isinstance(d, datetime):
        return d.date().isoformat()
    return d.isoformat()

def fetch_open_meteo_daily_range(
    start_date: Union[str, date, datetime],
    end_date: Union[str, date, datetime],
    latitude: float = 54.3233,
    longitude: float = 10.1228,
    timezone: str = "Europe/Berlin",
    daily_vars: List[str] = None,
    max_retries: int = 3,
    timeout: float = 15.0,
    use_cache: bool = True
) -> pd.DataFrame:
    """
    Fetch daily historical variables from Open-Meteo archive API for a given inclusive date range.
    By default uses Kiel coordinates (lat=54.3233, lon=10.1228).
    daily_vars example: ['precipitation_hours', 'sunshine_duration', 'rain_sum']
    Returns a pandas.DataFrame indexed by date (datetime.date) with columns named after daily_vars.
    """
    if daily_vars is None:
        daily_vars = ['precipitation_hours', 'sunshine_duration', 'rain_sum']

    start_iso = _iso_date(start_date)
    end_iso = _iso_date(end_date)
    start_dt = datetime.fromisoformat(start_iso).date()
    end_dt = datetime.fromisoformat(end_iso).date()
    if end_dt < start_dt:
        raise ValueError("end_date must be >= start_date")

    # Build cache key based on parameters and requested range
    key = f"{latitude:.6f}_{longitude:.6f}_{start_iso}_{end_iso}_{','.join(daily_vars)}_{timezone}"
    if use_cache and key in _OPEN_METEO_CACHE:
        df = _OPEN_METEO_CACHE[key].copy()
        return df

    base_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_iso,
        "end_date": end_iso,
        "daily": ",".join(daily_vars),
        "timezone": timezone
    }

    last_exc = None
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(base_url, params=params, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            daily = data.get("daily", {})
            times = daily.get("time", []) or []
            if not times:
                raise ValueError("No 'time' in API response daily block.")
            results = {"date": pd.to_datetime(times).date}
            for v in daily_vars:
                vals = daily.get(v, None)
                if vals is None:
                    # If a requested variable is missing, fill with NaN
                    results[v] = [np.nan] * len(times)
                else:
                    # Convert to numeric (floats), keep NaN if parsing fails
                    results[v] = [float(x) if x is not None else np.nan for x in vals]
            df = pd.DataFrame(results, index=pd.to_datetime(times).date)
            df.index.name = "date"
            df = df[[v for v in daily_vars]]  # ensure column order
            if use_cache:
                _OPEN_METEO_CACHE[key] = df.copy()
            return df
        except Exception as e:
            last_exc = e
            if attempt < max_retries:
                time.sleep(0.5 * attempt)
                continue
            raise RuntimeError(f"Failed fetching Open-Meteo archive: {e}") from e


In [ ]:
min_date = df_no_weather_code['Datum'].min()
max_date = df_no_weather_code['Datum'].max()
##min_date = "2013-07-01"
##max_date = "2014-07-30"
print (f"Fetching weather data from {min_date} to {max_date}...")
lat_kiel, lon_kiel = 54.3233, 10.1228
daily_vars = ['precipitation_hours', 'sunshine_duration', 'rain_sum','temperature_2m_mean']
df_extra_weather = fetch_open_meteo_daily_range(min_date, max_date, latitude=lat_kiel, longitude=lon_kiel, daily_vars=daily_vars)
# print columns of df_extended_weather


In [ ]:
df_extra_weather['sunshine_hours'] = df_extra_weather['sunshine_duration'] / 3600.0
#df_extended_weather.head()

# print row count for df_extended_weather and df_no_weather_code
print(f"Extended Weather Data Rows: {len(df_extra_weather)}")
print(f"Sales Data Rows: {len(df_no_weather_code)}")


### Extended Data Frames
Its with extra weather data from open meteo api: df_merged_extended_weather
with rolling 7 day average as well for umsatz

In [ ]:
df_extra_weather.index = pd.to_datetime(df_extra_weather.index)
df_merged_extended_weather = df_no_weather_code.merge(
    df_extra_weather,
    left_on='Datum',
    right_index=True,
    how='left'
)

print(f"Merged DataFrame Rows: {len(df_merged_extended_weather)}")
print(f"Merged DataFrame Shape: {df_merged_extended_weather.shape}")

df_merged_extended_weather = df_merged_extended_weather.dropna(subset=["id_umsatz"])
df_merged_extended_weather["id_umsatz"] = df_merged_extended_weather["id_umsatz"].astype("int64")
df_merged_extended_weather = df_merged_extended_weather.dropna(subset=["KielerWoche_kiwo"])
df_merged_extended_weather["KielerWoche_kiwo"] = df_merged_extended_weather["KielerWoche_kiwo"].astype("int64")

df_merged_extended_weather.head()

# Testing Regression

In [104]:
df_merged_extended_weather.tail()

#show only columns Datum,temperature_weather, temperature_2m_mean
#df_merged_extended_weather[['Datum', 'Temperatur_weather', 'temperature_2m_mean']].sample(10)

#calculate sample mean between temperature_weather and temperature_2m_mean
#df_merged_extended_weather['temperature_mean'] = df_merged_extended_weather[['Temperatur_weather', 'temperature_2m_mean']].mean(axis=1)
#df_merged_extended_weather['temperature_median'] = df_merged_extended_weather[['Temperatur_weather', 'temperature_2m_mean']].median(axis=1)
#df_merged_extended_weather[['Datum', 'Temperatur_weather', 'temperature_2m_mean','temperature_median','temperature_mean']].sample(10)

,Datum,KielerWoche_kiwo,Bewoelkung_weather,Temperatur_weather,Windgeschwindigkeit_weather,id_umsatz,Warengruppe_umsatz,Umsatz_umsatz,umsatz_rolling7,day_of_week,day,precipitation_hours,sunshine_duration,rain_sum,temperature_2m_mean,sunshine_hours,temp_diff,temperature_avg,temperature_mean,temperature_median
9761,2018-07-31,0,5.0,31.2875,11.0,1807311,1.0,123.075682,164.321051,1,Tuesday,0.0,46462.85,0.0,26.8,12.906347,4.4875,29.04375,29.04375,29.04375
9762,2018-07-31,0,5.0,31.2875,11.0,1807312,2.0,586.081666,656.898742,1,Tuesday,0.0,46462.85,0.0,26.8,12.906347,4.4875,29.04375,29.04375,29.04375
9763,2018-07-31,0,5.0,31.2875,11.0,1807313,3.0,285.872616,350.132441,1,Tuesday,0.0,46462.85,0.0,26.8,12.906347,4.4875,29.04375,29.04375,29.04375
9764,2018-07-31,0,5.0,31.2875,11.0,1807314,4.0,57.102795,74.455178,1,Tuesday,0.0,46462.85,0.0,26.8,12.906347,4.4875,29.04375,29.04375,29.04375
9765,2018-07-31,0,5.0,31.2875,11.0,1807315,5.0,261.341313,295.281152,1,Tuesday,0.0,46462.85,0.0,26.8,12.906347,4.4875,29.04375,29.04375,29.04375


In [ ]:
import scipy.stats as stats
import seaborn as sns
# Testing Regression
reg_cols=['Umsatz_umsatz','umsatz_rolling7','KielerWoche_kiwo','Bewoelkung_weather', 'rain_sum', 'sunshine_hours','Temperatur_weather','Windgeschwindigkeit_weather','precipitation_hours','day_of_week']
sns.pairplot(df_merged_extended_weather[reg_cols].dropna())
df_merged_extended_weather[reg_cols].corr()

In [ ]:
import statsmodels.formula.api as smf

# Example regression formula (adjust as needed)
formula = "Umsatz_umsatz ~  Temperatur_weather + Bewoelkung_weather  + C(Warengruppe_umsatz)"

# Drop rows with missing values in the regression columns
regression_df = df_merged_extended_weather.dropna(subset=[
    "Umsatz_umsatz", "Bewoelkung_weather", "rain_sum", "sunshine_hours",
    "Temperatur_weather", "Windgeschwindigkeit_weather", "precipitation_hours", "day_of_week","Warengruppe_umsatz"
])

# Fit the model
model = smf.ols(formula, data=regression_df).fit()

# Show summary
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Umsatz_umsatz   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     1662.
Date:                Tue, 02 Dec 2025   Prob (F-statistic):               0.00
Time:                        23:52:24   Log-Likelihood:                -53825.
No. Observations:                9334   AIC:                         1.077e+05
Df Residuals:                    9319   BIC:                         1.078e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       45.2595      4.216     10.735      0.000      36.995      53.524
C(day_of_week)[T.1]             -5.7340      2.996     -1.914      0.056     -11.606       0.138
C(day_of_week)[T.2]             -6.8894      2.998     -2.298      0.022     -12.766      -1.013
C(day_of_week)[T.3]             -0.2846      3.002     -0.095      0.924      -6.169       5.600
C(day_of_week)[T.4]              2.1018      3.015      0.697      0.486      -3.808       8.012
C(day_of_week)[T.5]             48.1065      3.002     16.023      0.000      42.221      53.992
C(day_of_week)[T.6]             53.4844      2.998     17.840      0.000      47.608      59.361
C(Warengruppe_umsatz)[T.2.0]   280.3569      2.565    109.294      0.000     275.329     285.385
C(Warengruppe_umsatz)[T.3.0]    41.2014      2.565     16.062      0.000      36.173      46.230
C(Warengruppe_umsatz)[T.4.0]   -34.2041      2.584    -13.235      0.000     -39.270     -29.138
C(Warengruppe_umsatz)[T.5.0]   154.6590      2.565     60.292      0.000     149.631     159.687
C(Warengruppe_umsatz)[T.6.0]   -29.1904      4.946     -5.902      0.000     -38.885     -19.495
Temperatur_weather               3.5575      0.142     25.018      0.000       3.279       3.836
sunshine_hours                   1.7347      0.254      6.840      0.000       1.238       2.232
Bewoelkung_weather               1.6174      0.400      4.042      0.000       0.833       2.402
==============================================================================
Omnibus:                     9598.960   Durbin-Watson:                   1.668
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2270364.828
Skew:                           4.607   Prob(JB):                         0.00
Kurtosis:                      78.847   Cond. No.                         130.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Merging test data from test csv


In [100]:
# Read test.csv into testdf
df_test = pd.read_csv("data/test.csv")

# Quick checks
print("Shape:", df_test.shape)
display(df_test.head())
display(df_test.tail())

Shape: (1830, 3)


,id,Datum,Warengruppe
0,1808011,2018-08-01,1
1,1808021,2018-08-02,1
2,1808031,2018-08-03,1
3,1808041,2018-08-04,1
4,1808051,2018-08-05,1


,id,Datum,Warengruppe
1825,1812226,2018-12-22,6
1826,1812236,2018-12-23,6
1827,1812246,2018-12-24,6
1828,1812276,2018-12-27,6
1829,1812286,2018-12-28,6


In [ ]:
## add weather Temperatur_weather from df_weather
df_test['Datum'] = pd.to_datetime(df_test['Datum'], errors='coerce').dt.normalize()
df_weather['Datum'] = pd.to_datetime(df_weather['Datum'], errors='coerce').dt.normalize()


# Left-join temperature from df_weather onto the test DataFrame (keep all test rows)
df_test_weather = df_test.merge(
    df_weather[['Datum', 'Temperatur_weather', 'Bewoelkung_weather']],
    on='Datum',
    how='left'
)

# now adding day_of_week column
df_test_weather['day_of_week'] = df_test_weather['Datum'].dt.weekday

# filling sunshine_hours from meteo api
test_min_date = df_test_weather['Datum'].min()
test_max_date = df_test_weather['Datum'].max()
print (f"Fetching weather data from {test_min_date} to {test_max_date}...")
lat_kiel, lon_kiel = 54.3233, 10.1228
daily_vars = ['sunshine_duration']
df_sunshine_date = fetch_open_meteo_daily_range(test_min_date, test_max_date, daily_vars=daily_vars) #defaults to kiel lat/lon
df_sunshine_date['sunshine_hours'] = df_sunshine_date['sunshine_duration'] / 3600.0

# rename df_sunshine_date column date to Datum for merging
# drop sunshine_duration column
df_sunshine_date.drop(columns=['sunshine_duration'],inplace=True)
df_sunshine_date = df_sunshine_date.rename_axis('Datum').reset_index()

# convert Datum to type datetime
df_sunshine_date['Datum'] = pd.to_datetime(df_sunshine_date['Datum'], errors='coerce').dt.normalize()
display(df_sunshine_date.head())

# merge sunshine_hours into df_test_weather
df_test_weather = df_test_weather.merge(
    df_sunshine_date[['Datum', 'sunshine_hours']],
    on='Datum',
    how='left'
)



# Quick sanity checks
print("Shape:", df_test_weather.shape)
print("Missing Temperatur_weather:", df_test_weather['Temperatur_weather'].isna().sum())
#check missing sunshine_hours
print("Missing sunshine_hours:", df_test_weather['sunshine_hours'].isna().sum())
#check missing Bewoelkung_weather
print("Missing Bewoelkung_weather:", df_test_weather['Bewoelkung_weather'].isna().sum())

display(df_test_weather.head())
display(df_test_weather.tail())

Fetching weather data from 2018-08-01 00:00:00 to 2019-07-30 00:00:00...


,Datum,sunshine_hours
0,2018-08-01,14.011489
1,2018-08-02,13.749917
2,2018-08-03,14.378883
3,2018-08-04,11.000000
4,2018-08-05,10.427422


Shape: (1830, 7)
Missing Temperatur_weather: 65
Missing sunshine_hours: 0
Missing Bewoelkung_weather: 65


,id,Datum,Warengruppe,Temperatur_weather,Bewoelkung_weather,day_of_week,sunshine_hours
0,1808011,2018-08-01,1,23.762500,0.0,2,14.011489
1,1808021,2018-08-02,1,26.187500,0.0,3,13.749917
2,1808031,2018-08-03,1,27.662500,1.0,4,14.378883
3,1808041,2018-08-04,1,25.137500,4.0,5,11.000000
4,1808051,2018-08-05,1,21.300000,7.0,6,10.427422
5,1808061,2018-08-06,1,23.100000,0.0,0,13.658097
6,1808071,2018-08-07,1,32.671428,0.0,1,14.067022
7,1808081,2018-08-08,1,28.150000,4.0,2,11.437142
8,1808091,2018-08-09,1,24.575000,5.0,3,9.198783
9,1808101,2018-08-10,1,17.325000,5.0,4,11.402433


,id,Datum,Warengruppe,Temperatur_weather,Bewoelkung_weather,day_of_week,sunshine_hours
1825,1812226,2018-12-22,6,4.3000,8.0,5,0.000000
1826,1812236,2018-12-23,6,6.4500,7.0,6,0.000000
1827,1812246,2018-12-24,6,2.5000,7.0,0,4.037928
1828,1812276,2018-12-27,6,7.1250,7.0,3,0.000000
1829,1812286,2018-12-28,6,7.3125,7.0,4,4.019717


In [ ]:
## interpolating missing Temperatur_weather
df_test_weather['Temperatur_weather'] = df_test_weather['Temperatur_weather'].interpolate()

## interpolate Bewoelkung_weather
df_test_weather['Bewoelkung_weather'] = df_test_weather['Bewoelkung_weather'].interpolate()

#15.0
#NaN
#17.0
# interpolation should fill the NaN with 16.0

print("Missing Temperatur_weather:", df_test_weather['Temperatur_weather'].isna().sum())
print("Missing Bewoelkung_weather:", df_test_weather['Bewoelkung_weather'].isna().sum())



## Predicting with df_test_weather

In [121]:
#predicting with df_test_weather
# rename df_test_weather.Warengruppe to Warengruppe_umsatz for consistency
df_test_weather = df_test_weather.rename(columns={"Warengruppe": "Warengruppe_umsatz"})

# Required columns for this model
req_cols = ['Temperatur_weather', 'Warengruppe_umsatz']

# Sanity checks
missing = [c for c in req_cols if c not in df_test_weather.columns]
if missing:
    raise ValueError(f"Missing columns for prediction: {missing}")

# Ensure numeric temperature
df_test_weather['Temperatur_weather'] = pd.to_numeric(df_test_weather['Temperatur_weather'], errors='coerce')





# Drop rows missing predictor values
#df_predicted = df_test_weather.dropna(subset=req_cols).copy()
df_predicted = df_test_weather.copy()



# Predict in next cell
#df_predicted['predicted_Umsatz_umsatz'] = model.predict(df_predicted)

print(df_predicted.shape)
display(df_predicted.head())
display(df_predicted.tail())





(1830, 7)


,id,Datum,Warengruppe_umsatz,Temperatur_weather,Bewoelkung_weather,day_of_week,sunshine_hours
0,1808011,2018-08-01,1,23.7625,0.0,2,14.011489
1,1808021,2018-08-02,1,26.1875,0.0,3,13.749917
2,1808031,2018-08-03,1,27.6625,1.0,4,14.378883
3,1808041,2018-08-04,1,25.1375,4.0,5,11.000000
4,1808051,2018-08-05,1,21.3000,7.0,6,10.427422


,id,Datum,Warengruppe_umsatz,Temperatur_weather,Bewoelkung_weather,day_of_week,sunshine_hours
1825,1812226,2018-12-22,6,4.3000,8.0,5,0.000000
1826,1812236,2018-12-23,6,6.4500,7.0,6,0.000000
1827,1812246,2018-12-24,6,2.5000,7.0,0,4.037928
1828,1812276,2018-12-27,6,7.1250,7.0,3,0.000000
1829,1812286,2018-12-28,6,7.3125,7.0,4,4.019717


Will Predict here

In [122]:

#predicting with df_test_weather
df_predicted['predicted_Umsatz_umsatz'] = model.predict(df_predicted)

print(df_predicted.shape)
display(df_predicted.head())
display(df_predicted.tail())

(1830, 8)


,id,Datum,Warengruppe_umsatz,Temperatur_weather,Bewoelkung_weather,day_of_week,sunshine_hours,predicted_Umsatz_umsatz
0,1808011,2018-08-01,1,23.7625,0.0,2,14.011489,147.211896
1,1808021,2018-08-02,1,26.1875,0.0,3,13.749917,161.989991
2,1808031,2018-08-03,1,27.6625,1.0,4,14.378883,172.332183
3,1808041,2018-08-04,1,25.1375,4.0,5,11.000000,208.344699
4,1808051,2018-08-05,1,21.3000,7.0,6,10.427422,203.929474


,id,Datum,Warengruppe_umsatz,Temperatur_weather,Bewoelkung_weather,day_of_week,sunshine_hours,predicted_Umsatz_umsatz
1825,1812226,2018-12-22,6,4.3000,8.0,5,0.000000,92.411832
1826,1812236,2018-12-23,6,6.4500,7.0,6,0.000000,103.821011
1827,1812246,2018-12-24,6,2.5000,7.0,0,4.037928,43.289257
1828,1812276,2018-12-27,6,7.1250,7.0,3,0.000000,52.453423
1829,1812286,2018-12-28,6,7.3125,7.0,4,4.019717,62.480038


## generating data submission csv from df_predicted
Will write a csv file in data named predicted.csv with these columns id,umsatz

In [123]:
# Prepare output frame and clean missing rows
df_out_predicted = df_predicted[["id", "predicted_Umsatz_umsatz"]].copy()
#rename columns accoding to kaggle submission requirements
df_out_predicted.rename(columns={ "predicted_Umsatz_umsatz": 'umsatz'}, inplace=True)

#print columns name df_out_predicted
print(df_out_predicted.columns)
print(df_out_predicted.shape)

# write to csv
df_out_predicted.to_csv("data/predicted.csv", index=False)
print("Wrote data/predicted.csv with", len(df_out_predicted), "rows.")




Index(['id', 'umsatz'], dtype='object')
(1830, 2)
Wrote data/predicted.csv with 1830 rows.
